### Если юзер Android нажимает “отменить”, предлагать промокод в случае НЕ отмены
### Ожидаем: ниже отмены на Android устройствах

In [1]:
import pandas as pd
import numpy as np
import scipy
from scipy import stats

In [2]:
data = pd.read_csv('post_abtest_data.csv')
data[['date','user_id','user_cancelled','groups','os']]

,date,user_id,user_cancelled,groups,os
0,2020-05-28,867689,0.0,Control,Android
1,2020-05-29,867689,NaN,Control,iOS
2,2020-05-25,867689,0.0,Control,Android
3,2020-05-29,752172,0.0,Control,Android
4,2020-05-28,752172,0.0,Control,iOS
...,...,...,...,...,...
101495,2020-05-25,777065,NaN,Control,Android
101496,2020-05-28,340213,0.0,Control,iOS
101497,2020-05-26,352605,0.0,Test,Android
101498,2020-05-26,216082,NaN,Control,Android


In [103]:
 # только Android
os = data[data['os'] == 'Android']
os = os[['date','user_id','user_cancelled','groups']]

In [104]:
os.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60825 entries, 0 to 101499
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   date            60825 non-null  object 
 1   user_id         60825 non-null  int64  
 2   user_cancelled  42177 non-null  float64
 3   groups          60825 non-null  object 
dtypes: float64(1), int64(1), object(2)
memory usage: 2.3+ MB


In [105]:
os.fillna(method='ffill',inplace=True)# замена пропусков следующим значением
os.head()

,date,user_id,user_cancelled,groups
0,2020-05-28,867689,0.0,Control
2,2020-05-25,867689,0.0,Control
3,2020-05-29,752172,0.0,Control
5,2020-05-25,486559,0.0,Control
7,2020-05-30,486559,0.0,Control


In [106]:
# использую функцию для расчета t критерия м/у тестовой и контрольной группой
def student(param):
    return stats.ttest_ind(os[os['groups']=='Test'].user_cancelled,os[os['groups']=='Control'].user_cancelled)[1]

In [107]:
date_ = os.groupby(['date']).agg({'user_id':'count','user_cancelled':'mean'}) # группирую по дате
date_

,user_id,user_cancelled
date,,
2020-05-25,8597,0.169710
2020-05-26,8629,0.159231
2020-05-27,8662,0.164512
2020-05-28,8646,0.171293
2020-05-29,8721,0.165921
2020-05-30,8766,0.163130
2020-05-31,8804,0.158110


In [108]:
date_['p_value']=date_['user_cancelled'].apply(student) #добавляю столбец со значением 
date_

,user_id,user_cancelled,p_value
date,,,
2020-05-25,8597,0.169710,1.598509e-33
2020-05-26,8629,0.159231,1.598509e-33
2020-05-27,8662,0.164512,1.598509e-33
2020-05-28,8646,0.171293,1.598509e-33
2020-05-29,8721,0.165921,1.598509e-33
2020-05-30,8766,0.163130,1.598509e-33
2020-05-31,8804,0.158110,1.598509e-33
